# Training Model for MCS

In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd

features = ['fio2','peep', 'inspiratory_pressure', 'respiration_rate', 'flow', 'tidal_volume', 'left_severity', 'right_severity', 'brain_injury_severity', 
            'Mode_ACP', 'Mode_ACV', 'Mode_CMP', 'Mode_CMV']

targets = ['Respiratory Rate', 'Heart Rate', 'SpO2', 'PaO2', 'Systolic BP',
           'Diastolic BP', 'MAP', 'PIP', 'pH', 'PaCo2', 'Hematocrit', 'Hemoglobin']

df_model = pd.read_csv("model_data.csv")
# === 1. Prepare data ===
X = df_model[features]
Y = df_model[targets]

# Split train-test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=42)

In [2]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


best_params = {
    'n_estimators': 2261,
    'max_depth': 64,
    'min_samples_split': 3,
    'min_samples_leaf': 1,
    'max_features': 0.8588789126735755
}

rf = RandomForestRegressor(
    random_state=42,
    n_jobs=-1,
    **best_params
)

model = MultiOutputRegressor(rf)

best_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('model', model)
])

best_pipe.fit(X_train, Y_train)


,steps,"[('scaler', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,copy,True
,with_mean,True
,with_std,True
,estimator,RandomForestR...ndom_state=42)
,n_jobs,None
,n_estimators,2261
,criterion,'squared_error'


In [13]:
import joblib 

joblib.dump(best_pipe, 'RFR_model.joblib') 

['RFR_model.joblib']

In [3]:
import numpy as np

def run_mc_simulation_uniform(
        left_sev: float,
        right_sev: float,
        brain_sev: float,
        bounds: dict,
        N: int = 10000,
        best_pipe=None
    ):
    """
    Monte Carlo with:
      - uniform sampling inside manual bounds
      - mode-specific feature activation:
           * pressure-control: inspiratory_pressure
           * volume-control: flow, tidal_volume
      - shared features: peep, fio2, respiration_rate
    """

    continuous_features = list(bounds.keys())
    mode_features = ['Mode_ACP','Mode_ACV','Mode_CMP','Mode_CMV']

    # --- Mode-specific feature sets ---
    pressure_feats = ['inspiratory_pressure', 'peep', 'fio2', 'respiration_rate']
    volume_feats   = ['flow', 'tidal_volume', 'peep', 'fio2', 'respiration_rate']

    # All non-mode features (used to fill others with zeros)
    all_core_feats = set(continuous_features)

    N_per_mode = N // 4
    mc_blocks = []

    for mode in mode_features:

        block_data = {}

        if mode in ['Mode_ACP', 'Mode_CMP']:  # PRESSURE modes
            active_feats = pressure_feats
        else:                                  # VOLUME modes
            active_feats = volume_feats

        # Sample only active features
        for feat in active_feats:
            low, high = bounds[feat]
            block_data[feat] = np.random.uniform(low, high, N_per_mode)

        # Set inactive features to zero
        inactive_feats = list(all_core_feats - set(active_feats))
        for feat in inactive_feats:
            block_data[feat] = np.zeros(N_per_mode)

        # Build block DataFrame
        block = pd.DataFrame(block_data)

        # Fixed severities
        block['left_severity'] = left_sev
        block['right_severity'] = right_sev
        block['brain_injury_severity'] = brain_sev

        # One-hot encode modes
        for m in mode_features:
            block[m] = 1 if m == mode else 0

        mc_blocks.append(block)

    # Combine all
    mc_samples = pd.concat(mc_blocks, ignore_index=True)
    
    expected_features = [
    'fio2','peep','inspiratory_pressure','respiration_rate',
    'flow','tidal_volume',
    'left_severity','right_severity','brain_injury_severity',
    'Mode_ACP','Mode_ACV','Mode_CMP','Mode_CMV'
    ]

    mc_samples = mc_samples.reindex(columns=expected_features)


    # Predict
    vital_names = [
        "Respiratory Rate","Heart Rate","SpO2","PaO2","Systolic BP",
        "Diastolic BP","MAP","PIP","pH","PaCO2","Hematocrit","Hemoglobin"
    ]

    mc_predictions = best_pipe.predict(mc_samples)
    mc_predictions_df = pd.DataFrame(mc_predictions, columns=vital_names)

    return mc_samples, mc_predictions_df


In [8]:
# -------------------------------------------------------
# MANUAL PARAMETER BOUNDS (you can edit anytime)
# -------------------------------------------------------
manual_bounds = {
    'fio2':                (0.90, 1.0),
    'peep':                (10, 20),
    'inspiratory_pressure':(10, 35),
    'respiration_rate':    (8, 20),
    'flow':                (20, 80),
    'tidal_volume':        (200, 800)
}


In [9]:
samples, preds = run_mc_simulation_uniform(
    left_sev=0.8,
    right_sev=0.6,
    brain_sev=0.3,
    bounds = manual_bounds,
    N=10000,
    best_pipe=best_pipe
)

total = pd.concat([samples, preds], axis=1, ignore_index=False)
print(total.head())

       fio2       peep  inspiratory_pressure  respiration_rate  flow  \
0  0.963282  12.478151             29.370963          8.503119   0.0   
1  0.925009  11.459476             20.199673         13.776763   0.0   
2  0.981636  18.288773             12.522520         17.130492   0.0   
3  0.941293  18.035435             20.300351         19.279407   0.0   
4  0.948700  12.058150             18.272866         10.300798   0.0   

   tidal_volume  left_severity  right_severity  brain_injury_severity  \
0           0.0            0.8             0.6                    0.3   
1           0.0            0.8             0.6                    0.3   
2           0.0            0.8             0.6                    0.3   
3           0.0            0.8             0.6                    0.3   
4           0.0            0.8             0.6                    0.3   

   Mode_ACP  ...      SpO2        PaO2  Systolic BP  Diastolic BP        MAP  \
0         1  ...  0.967080  214.835953   109.693

# Scoring Function

In [6]:
import numpy as np
import pandas as pd

# ---------------------------
# Configuration: bounds, weights, penalties
# ---------------------------
# Clinical bounds table (from your text)
VITAL_BOUNDS = {
    "Heart Rate": (60, 100),        # beats/min
    "MAP": (70, 100),               # mmHg
    "Systolic BP": (90, 150),       # mmHg
    "Diastolic BP": (50, 90),       # mmHg
    "SpO2": (0.90, 1.00),           # 0..1
    "PaO2": (80, 400),              # mmHg
    "Respiratory Rate": (12, 24),   # breaths/min
    "pH": (7.40, 7.48),             # unitless
    "PaCo2": (35, 42),              # mmHg
    "Hemoglobin": (10, 16),         # g/dL
    "Haematocrit": (0.30, 0.48),     # 0..1
    "PIP" : (15, 25) ,              # cmH₂O
}

# Clinical weights (sum to 1.0)
VITAL_WEIGHTS = {
    "SpO2": 0.10,
    "PaO2": 0.15,
    "pH": 0.06,
    "PaCo2": 0.1,
    "MAP": 0.05,
    "Heart Rate": 0.15,
    "Respiratory Rate": 0.2,
    "PIP": 0.1,           # note: PIP may or may not be present in your df
    "Systolic BP": 0.06,
    "Diastolic BP": 0.04,
    "Hemoglobin": 0.05,
    "Haematocrit": 0.05
}
# If some keys in VITAL_WEIGHTS not in VITAL_BOUNDS (e.g., PIP), bounds must be added or weight removed.

# Asymmetric exponential penalty strengths (higher => stronger penalty)
# alpha = low-side penalty, beta = high-side penalty
# recommendation: alpha > beta for safety-critical low values
PENALTIES = {
    "SpO2":      {"alpha": 8.0,  "beta": 2.0},
    "PaO2":      {"alpha": 7.0,  "beta": 3.0},
    "pH":        {"alpha": 6.0, "beta": 4.0},
    "PaCo2":     {"alpha": 6.0,  "beta": 6.0},
    "MAP":       {"alpha": 6.0,  "beta": 1.5},
    "Heart Rate":{"alpha": 2.0,  "beta": 7},
    "Respiratory Rate": {"alpha": 2.0, "beta": 7},
    "PIP":       {"alpha": 1.5,  "beta": 6},
    "Systolic BP":{"alpha": 7, "beta": 1.5},
    "Diastolic BP":{"alpha": 6.0,"beta": 1.5},
    "Hemoglobin":{"alpha": 3, "beta": 1.0},
    "Haematocrit":{"alpha": 3, "beta": 1.0}
}

# ---------------------------
# Utility: per-variable compliance q_i(s)
# ---------------------------
def compliance_q(value, vmin, vmax, alpha=5.0, beta=2.0, eps=1e-12):
    """
    Asymmetric exponential compliance score q in (0,1].
      - q = 1 if vmin <= value <= vmax
      - if value < vmin: q = exp(-alpha * (vmin - value) / max(|vmin|, eps))
      - if value > vmax: q = exp(-beta  * (value - vmax) / max(|vmax|, eps))
    The division by vmin/vmax makes penalties relative (fractional deviation).
    eps prevents division by zero for bounds that can be zero (e.g., SpO2 lower bound 0.9 - not zero).
    """
    if pd.isna(value):
        return 0.0

    # inside range
    if (value >= vmin) and (value <= vmax):
        return 1.0

    # below lower bound -> stronger penalty
    if value < vmin:
        # relative deficit
        rel = (vmin - value) / max(abs(vmin), eps)
        q = float(np.exp(- alpha * rel))
        return q

    # above upper bound -> softer penalty
    rel = (value - vmax) / max(abs(vmax), eps)
    q = float(np.exp(- beta * rel))
    return q

# ---------------------------
# Compute composite score S_t for each row
# ---------------------------
def compute_physiological_score(df,
                                bounds=VITAL_BOUNDS,
                                weights=VITAL_WEIGHTS,
                                penalties=PENALTIES,
                                score_col_prefix="q_"):
    """
    Adds:
      - per-vital q columns named score_col_prefix + vital (spaces replaced by '__')
      - 'phys_score' column = normalized weighted sum (in [0,1])
    Returns a new DataFrame (copy).
    """
    df = df.copy()
    q_cols = []
    # ensure weights sum to 1 (normalize if not)
    w = weights.copy()
    total_w = sum(w.values())
    if abs(total_w - 1.0) > 1e-8:
        for k in w:
            w[k] = w[k] / total_w

    for vital, (vmin, vmax) in bounds.items():
        col_name = vital
        score_name = score_col_prefix + vital.replace(" ", "__")
        if col_name not in df.columns:
            # skip if missing in dataframe
            continue
        alpha = penalties.get(vital, {}).get("alpha", 5.0)
        beta  = penalties.get(vital, {}).get("beta", 2.0)
        df[score_name] = df[col_name].apply(lambda x: compliance_q(x, vmin, vmax, alpha=alpha, beta=beta))
        q_cols.append((vital, score_name))

    # compute weighted sum (use only vitals present)
    weighted_q = np.zeros(len(df), dtype=float)
    used_weights_sum = 0.0
    for vital, score_name in q_cols:
        wt = w.get(vital, 0.0)
        weighted_q += wt * df[score_name].fillna(0.0).values
        used_weights_sum += wt

    # If some weights were missing (used_weights_sum < 1) normalize by used sum
    if used_weights_sum > 0:
        df["phys_score"] = weighted_q / used_weights_sum
    else:
        df["phys_score"] = 0.0

    return df

# ---------------------------
# Strategy handling: define ventilator strategy key columns
# ---------------------------
STRATEGY_COLS = [
    "fio2",
    "peep",
    "inspiratory_pressure",
    "respiration_rate",
    "flow",
    "tidal_volume",
    "Mode_ACP", 
    "Mode_ACV", # remove if not present
    "Mode_CMP",
    "Mode_CMV"
]

# helper to create a strategy identifier string (readable)
def make_strategy_id(df_row, cols=STRATEGY_COLS):
    parts = []
    for c in cols:
        if c in df_row:
            parts.append(f"{c}={df_row[c]}")
    return "|".join(parts)

# ---------------------------
# Aggregation & top-k per severity
# ---------------------------
def topk_strategies_per_severity(df, k=3,
                                severity_cols=("left_severity","right_severity","brain_injury_severity"),
                                strategy_cols=STRATEGY_COLS,
                                score_col="phys_score"):
    """
    Returns a DataFrame with top-k strategy configurations per severity triple.
    For Monte Carlo data, input df will typically have many rows per unique strategy; we compute mean & std.
    Output columns:
      severity_cols..., strategy_cols..., mean_score, std_score, n_samples
    """
    # keep only rows that have required columns
    required = list(severity_cols) + strategy_cols + [score_col]
    present = [c for c in required if c in df.columns]
    # group by severity + strategy
    group_cols = [c for c in severity_cols if c in df.columns] + [c for c in strategy_cols if c in df.columns]
    if not group_cols:
        raise ValueError("No grouping columns present. Check severity_cols and strategy_cols vs dataframe columns.")

    g = df.groupby(group_cols)[score_col].agg(["mean","std","count"]).reset_index().rename(columns={"mean":"mean_score","std":"std_score","count":"n"})
    # sort by severity + mean_score desc, then pick top-k per severity keys
    severity_keys = [c for c in severity_cols if c in df.columns]
    sort_cols = severity_keys + ["mean_score"]
    g = g.sort_values(by=sort_cols, ascending= [True]*len(severity_keys) + [False]).reset_index(drop=True)

    # pick top-k
    topk = g.groupby(severity_keys).head(k).reset_index(drop=True)
    return topk

# ---------------------------
# High-level pipeline functions for deterministic and Monte Carlo
# ---------------------------
def evaluate_deterministic(df_det, **kwargs):
    """
    df_det: deterministic dataset (one or more rows per strategy)
    Returns:
      - df_scored: dataframe with phys_score added
      - top3: top-3 strategies per severity (mean phys_score)
    """
    df_scored = compute_physiological_score(df_det, **kwargs)
    top3 = topk_strategies_per_severity(df_scored, k=3)
    return df_scored, top3

def evaluate_montecarlo(df_mc, k=3, mode_col=None, **kwargs):
    """
    df_mc: Monte Carlo runs (many rows). If you have 'mode' column (e.g., ventilation mode),
           you can pass mode_col="Mode" to get top-k per severity per mode.
    Returns:
      - df_scored: dataframe with phys_score added
      - topk_overall: top-k strategies per severity (across all modes)
      - optionally dict of top-k per mode if mode_col is provided
    """
    df_scored = compute_physiological_score(df_mc, **kwargs)
    topk_overall = topk_strategies_per_severity(df_scored, k=k)
    per_mode = None
    if (mode_col is not None) and (mode_col in df_scored.columns):
        per_mode = {}
        modes = df_scored[mode_col].unique()
        for m in modes:
            dfm = df_scored[df_scored[mode_col] == m]
            per_mode[m] = topk_strategies_per_severity(dfm, k=k)
    return df_scored, topk_overall, per_mode

# ---------------------------
# Example usage
# ---------------------------
# if __name__ == "__main__":
#     # Example: load deterministic CSV
#     # df_det = pd.read_csv("deterministic_runs.csv")
#     # df_det = df_model.copy()  # replace with your deterministic DataFrame
#     # df_scored_det, top3_det = evaluate_deterministic(df_det)
#     # print("Top3 deterministic strategies per severity:")
#     # print(top3_det)

#     # # Example: load monte carlo CSV (10000 rows, 2500 per mode if you have 4 modes)
#     # result_monte_carlo = pd.read_csv("D:/BTP_Shivam/After_simulations/Machine_learning_with_36h/k-fold/monte_carlo_balanced_results.csv")  # replace with your Monte Carlo DataFrame
#     # df_mc = result_monte_carlo.copy()
#     # df_scored_mc, top3_mc, per_mode_top3 = evaluate_montecarlo(df_mc, k=3, mode_col="Mode")
#     # print("Top3 Monte Carlo strategies per severity (overall):")
#     # print(top3_mc)
#     # # if per_mode_top3 is not None:
#     # #     for m, table in per_mode_top3.items():
#     # #         print(f"Mode: {m}")
#     # #         print(table)
#     pass


In [10]:
df_mc = total.copy()
df_scored_mc, top3_mc, per_mode_top3 = evaluate_montecarlo(df_mc, k=3, mode_col="Mode")
print("Top3 Monte Carlo strategies per severity (overall):")
print(top3_mc)

Top3 Monte Carlo strategies per severity (overall):
   left_severity  right_severity  brain_injury_severity      fio2       peep  \
0            0.8             0.6                    0.3  0.901695  10.140768   
1            0.8             0.6                    0.3  0.902551  13.970561   
2            0.8             0.6                    0.3  0.902691  12.696682   

   inspiratory_pressure  respiration_rate       flow  tidal_volume  Mode_ACP  \
0                   0.0         18.389477  75.667213    662.591941         0   
1                   0.0         18.084442  63.933043    691.484281         0   
2                   0.0         19.149147  64.470650    608.442007         0   

   Mode_ACV  Mode_CMP  Mode_CMV  mean_score  std_score  n  
0         0         0         1    0.895588        NaN  1  
1         0         0         1    0.895588        NaN  1  
2         0         0         1    0.895588        NaN  1  


In [11]:
df_scored_mc.head()

,fio2,peep,inspiratory_pressure,respiration_rate,flow,tidal_volume,left_severity,right_severity,brain_injury_severity,Mode_ACP,...,q_MAP,q_Systolic__BP,q_Diastolic__BP,q_SpO2,q_PaO2,q_Respiratory__Rate,q_pH,q_Hemoglobin,q_PIP,phys_score
0,0.963282,12.478151,29.370963,8.503119,0.0,0.0,0.8,0.6,0.3,1,...,1.0,1.0,1.0,1.0,1.0,0.036556,0.700249,2.427543e-19,1.0,0.588922
1,0.925009,11.459476,20.199673,13.776763,0.0,0.0,0.8,0.6,0.3,1,...,1.0,1.0,1.0,1.0,1.0,0.036556,0.700249,2.427543e-19,1.0,0.588922
2,0.981636,18.288773,12.522520,17.130492,0.0,0.0,0.8,0.6,0.3,1,...,1.0,1.0,1.0,1.0,1.0,0.041843,0.562331,2.509147e-19,1.0,0.575430
3,0.941293,18.035435,20.300351,19.279407,0.0,0.0,0.8,0.6,0.3,1,...,1.0,1.0,1.0,1.0,1.0,0.049208,0.701495,2.428169e-19,1.0,0.591583
4,0.948700,12.058150,18.272866,10.300798,0.0,0.0,0.8,0.6,0.3,1,...,1.0,1.0,1.0,1.0,1.0,0.036556,0.700249,2.427543e-19,1.0,0.588922


In [12]:
print(top3_mc)

   left_severity  right_severity  brain_injury_severity      fio2       peep  \
0            0.8             0.6                    0.3  0.901695  10.140768   
1            0.8             0.6                    0.3  0.902551  13.970561   
2            0.8             0.6                    0.3  0.902691  12.696682   

   inspiratory_pressure  respiration_rate       flow  tidal_volume  Mode_ACP  \
0                   0.0         18.389477  75.667213    662.591941         0   
1                   0.0         18.084442  63.933043    691.484281         0   
2                   0.0         19.149147  64.470650    608.442007         0   

   Mode_ACV  Mode_CMP  Mode_CMV  mean_score  std_score  n  
0         0         0         1    0.895588        NaN  1  
1         0         0         1    0.895588        NaN  1  
2         0         0         1    0.895588        NaN  1  
